# Assignment 2
You will continue to work with the hue data files supplied for Assignment 1. We assume that the folder that you work in has the following structure.
<code>
assignment02.ipynb
hue_upload.csv
hue_upload2.csv
</code>

The first four columns represent the `row id`, `user id`, `event id`, and `value`. Any extra columns are irrelevant. For example, the first row of one file reads:

`"1";"10";"lamp change 29 mei 2015 19 08 33 984";"OFF"`

As you can see, the `event id` encompasses both a description of the event (`lamp_change`) and the date/time
(May 29, 7:08:33 pm). The following events are considered informative:

| String               | Description                                               |
-----------------------|------------------------------------------------------------
| `lamp_change`          | Light control via app                                     |
| `nudge_time`           | Automatic light dim time for people in experimental group |
| `bedtime_tonight`      | Intended bedtime (self-reported)                          |
| `risetime`             | Rise time (self-reported)                                 |
| `rise_reason`          | Reason for rising (self-reported)                         |
| `adherence importance` | Adherence (self-reported)                                 |
| `fitness`              | Fitness (self-reported)                                   |

All self-reported values are entered around noon. Records with other events may be ignored.

In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import pymongo
import re 

pd.options.display.max_rows = 20

ModuleNotFoundError: No module named 'pymongo'

In [2]:
# My locale was set to English so did not understand the dates in Dutch
import locale
locale.setlocale(locale.LC_TIME, 'nl_NL.utf8')

'nl_NL.utf8'

## Exercise 1 (70 points)
The first part of this assignment is to write a Python function `read_csv_data` that reads the data into
a Pandas DataFrame. The index should be a (date, user) tuple, where date is stored in datetime.datetime format
(see the document with the "Tips"). The columns of your Pandas DataFrame should be `bedtime`, `intended_bedtime`,
`rise_time`, `rise_reason`, `fitness`, `adherence_importance` and `in_experimental_group`. Note: it is important to stick to this nomenclature. The way to do this is by going through the CSV data line by line, and parsing each line
individually, following these requirements:

<ul>
<li>
`bedtime` should be inferred from the `lamp_change` event. There are multiple reasonable ways to accomplish this. In this assigment, we define the  bedtime as the last OFF state of the `lamp_change` in the interval between 7 pm of the current day and 6 am of the next day. For example, from the row printed above you may
infer that the person did not sleep before 7:08:33 pm. As you go through the lines in the csv file, whenever
you discover new relevant information, you either update an existing record in the dataframe (if a record
for that day and user exists), or you create a new record (see the document with the "Tips").
<br><br>
For example, if you encounter a line where user 10 turns the light on at 9 pm and another line where he
turns it off at 10 pm (still on May 29), you update the record above to change the bedtime to 10 pm. If
someone falls asleep past midnight, the bedtime should be stored in the record corresponding to the day before. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>`intended_bedtime` should be filled in based on the `bedtime_tonight` event. Note that 1030 probably means 10:30 in the evening. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>
`rise_time`. The value for the column in your solution should be obtained from the `risetime` event in the CSV file.
<br><br></li>
<li>
`rise_reason`, `fitness` and `adherence_importance` values should be copied from the CSV file. Note that if
multiple distinct values are entered, the last should be assumed to be correct.
<br><br></li>
<li>
`in_experimental_group` should be boolean (True/False). The default value is False, but should be changed
to True if a `nudge_time` event is encountered. If a user is in the experimental group on one day, he is on
all days.
</li>
</ul>    

In [ ]:
def read_csv_data(filenames):
# YOUR CODE HERE

# YOUR CODE ENDS HERE

In [3]:
def read_csv_data(filenames):
# YOUR CODE HERE
    dataframes = []
    colnames = ['row_id', 'user_id', 'event_id', 'value']
    for filename in filenames:
        dataframes.append(pd.read_csv(filename, sep = ';', names = colnames, header = None, encoding ='utf_8'))
        combined_df = pd.concat(dataframes)
    return combined_df
# YOUR CODE ENDS HERE

In [4]:
df = read_csv_data(['hue_upload.csv', 'hue_upload2.csv'])
df

,row_id,user_id,event_id,value
0,1,10,lamp_change_29_mei_2015_19_08_33_984,OFF
1,2,10,0010_31_mei_2015_bedtime_tonight,2300
2,3,10,0010_31_mei_2015_fitness,52
3,4,10,morning_backup_minute,0
4,5,10,lamp_change_29_mei_2015_19_08_33_942,OFF
...,...,...,...,...
1134006,1134007,63,error_event_15_september_2015_20_33_31_287,1_lamps_found
1134007,1134008,63,error_event_16_september_2015_21_15_35_568,0_lamps_found
1134008,1134009,63,start_experiment,2015-09-06T00:00:00.000+02:00
1134009,1134010,63,lamp_change_09_september_2015_23_39_52_045,OFF


In [5]:
# Pulling out 'Date' from the Event_ID column
df["date_event_id"] = df["event_id"].str.findall(r"_\d+_\w+_\d{4}").astype(str).str.replace('\[|\]|\'', '')
df["date_event_id"] = df["date_event_id"].str.replace('_'," ").str.strip()
df.replace(r'^\s*$', np.nan, regex=True, inplace = True) #replacing whitespace with NaN
df

,row_id,user_id,event_id,value,date_event_id
0,1,10,lamp_change_29_mei_2015_19_08_33_984,OFF,29 mei 2015
1,2,10,0010_31_mei_2015_bedtime_tonight,2300,31 mei 2015
2,3,10,0010_31_mei_2015_fitness,52,31 mei 2015
3,4,10,morning_backup_minute,0,NaN
4,5,10,lamp_change_29_mei_2015_19_08_33_942,OFF,29 mei 2015
...,...,...,...,...,...
1134006,1134007,63,error_event_15_september_2015_20_33_31_287,1_lamps_found,15 september 2015
1134007,1134008,63,error_event_16_september_2015_21_15_35_568,0_lamps_found,16 september 2015
1134008,1134009,63,start_experiment,2015-09-06T00:00:00.000+02:00,NaN
1134009,1134010,63,lamp_change_09_september_2015_23_39_52_045,OFF,09 september 2015


In [6]:
# Pulling out 'Time' from the Event_ID column

df["time_event_id"] = df["event_id"].str.findall(r"_\d{2}_\d{2}_\d{2}").astype(str).str.replace('\[|\]|\'', '')
df["time_event_id"] = df["time_event_id"].str.replace('_'," ").str.strip()
df["time_event_id"] = df["time_event_id"].str.replace(" ",":")
df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
df["time_event_id"].fillna(value = "00:00:00", inplace = True) 

In [7]:
# Getting unique events from the 'event_id' column to isolate 'bedtimetonight' and 'risetime'

def remove_all_whitespace(text):
    sentence = ''.join(text.split())
    return sentence

df["event_id_clean"] = df["event_id"].str.findall(r"[a-zA-Z]+\w[a-zA-Z]+\w").astype(str).str.replace('\[|\]|\'', '').str.replace("_"," ")
df["event_id_clean"].replace(" ","")
df[['event1', 'event2']] = df["event_id_clean"].str.split(',',expand = True)

# Apply 'function' above
df.event2.fillna(value=np.nan, inplace=True)
df['event2'] = df.apply(lambda x: remove_all_whitespace(x['event2']) if x['event2'] is not np.nan else np.nan, axis=1)

In [8]:
df

,row_id,user_id,event_id,value,date_event_id,time_event_id,event_id_clean,event1,event2
0,1,10,lamp_change_29_mei_2015_19_08_33_984,OFF,29 mei 2015,19:08:33,"lamp change , mei",lamp change,mei
1,2,10,0010_31_mei_2015_bedtime_tonight,2300,31 mei 2015,00:00:00,"mei , bedtime tonight",mei,bedtimetonight
2,3,10,0010_31_mei_2015_fitness,52,31 mei 2015,00:00:00,"mei , fitness",mei,fitness
3,4,10,morning_backup_minute,0,NaN,00:00:00,"morning backup , minute",morning backup,minute
4,5,10,lamp_change_29_mei_2015_19_08_33_942,OFF,29 mei 2015,19:08:33,"lamp change , mei",lamp change,mei
...,...,...,...,...,...,...,...,...,...
1134006,1134007,63,error_event_15_september_2015_20_33_31_287,1_lamps_found,15 september 2015,20:33:31,"error event , september",error event,september
1134007,1134008,63,error_event_16_september_2015_21_15_35_568,0_lamps_found,16 september 2015,21:15:35,"error event , september",error event,september
1134008,1134009,63,start_experiment,2015-09-06T00:00:00.000+02:00,NaN,00:00:00,start experiment,start experiment,NaN
1134009,1134010,63,lamp_change_09_september_2015_23_39_52_045,OFF,09 september 2015,23:39:52,"lamp change , september",lamp change,september


In [9]:
# Extract all unique string values in the value column associated to 'risetime' and 'bedtimetonight' events
# This gives us the list of string values that we will need to convert to timestamps

# Create risetime list

bool_series = df["event2"] == 'risetime'
df[bool_series]
risetime_array = df[bool_series]['value'].unique()
risetime_array
#print(risetime_array)
risetime_list = risetime_array.tolist()
cleaned_risetime = [x for x in risetime_list if x == x]

# Create bedtimetonight list

bool_series = df["event2"] == 'bedtimetonight'
bedtimetonight_array = df[bool_series]['value'].unique()
#print(bedtimetonight_array)
bedtimetonight_list = bedtimetonight_array.tolist()
cleaned_bedtimetonight = [x for x in bedtimetonight_list if x == x]

# Combined list
timestamp_list = cleaned_risetime + cleaned_bedtimetonight
timestamp_list.sort()

timestamp_unique = [] 
[timestamp_unique.append(x) for x in timestamp_list if x not in timestamp_unique] 
timestamp_unique

['000',
 '0000',
 '0010',
 '0015',
 '0026',
 '0030',
 '0036',
 '0040',
 '0100',
 '0130',
 '01900',
 '0200',
 '0230',
 '030',
 '0300',
 '0330',
 '0400',
 '0530',
 '0552',
 '0553',
 '0600',
 '0620',
 '0630',
 '0638',
 '0645',
 '0700',
 '0709',
 '0710',
 '0715',
 '0722',
 '0727',
 '0728',
 '0730',
 '0734',
 '0735',
 '0738',
 '0740',
 '0745',
 '0750',
 '0752',
 '0753',
 '0757',
 '0800',
 '0804',
 '0810',
 '0815',
 '0819',
 '0820',
 '0822',
 '0825',
 '0830',
 '0831',
 '0834',
 '0835',
 '0840',
 '0846',
 '0850',
 '0900',
 '0915',
 '0920',
 '0930',
 '0945',
 '0950',
 '10',
 '100',
 '1000',
 '1006',
 '1015',
 '1030',
 '1100',
 '1101',
 '1115',
 '1120',
 '1130',
 '1135',
 '1145',
 '12',
 '1200',
 '1234',
 '1235',
 '1236',
 '1237',
 '1300',
 '1423',
 '1552',
 '1553',
 '1556',
 '1557',
 '1652',
 '1653',
 '1825',
 '2',
 '200',
 '22',
 '2200',
 '2215',
 '2230',
 '2240',
 '2245',
 '22454',
 '2250',
 '23',
 '230',
 '2300',
 '2305',
 '2315',
 '2330',
 '2340',
 '2345',
 '2350',
 '2359',
 '24',
 '2400',

In [60]:
# Create dictionary 'key' lists based on length of characters in string to evaluate how each should be treated

char5 = []
for item in timestamp_unique:
    text = item.strip()
    length = len(text)
    if length == 5:
        char5.append(text)
#print(char5)

char4 = []
for item in timestamp_unique:
    text = item.strip()
    length = len(text)
    if length == 4:
        char4.append(text)
#print(char4)

char3 = []
for item in timestamp_unique:
    text = item.strip()
    length = len(text)
    if length == 3:
        char3.append(text)
#print(char3)

char2 = []
for item in timestamp_unique:
    text = item.strip()
    length = len(text)
    if length == 2:
        char2.append(text)
#print(char2)

char1 = []
for item in timestamp_unique:
    text = item.strip()
    length = len(text)
    if length == 1:
        char1.append(text)
#print(char1)

In [61]:
# Create values (value list) for keys (char lists) to create dictionary

val5 = []
for item in char5:
    filled = item.zfill(6)
    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
    val5.append(new)
    
dic5 = dict(zip(char5,val5))

val4 = []
for item in char4:
    new = (':'.join(item[i:i+2] for i in range(0, len(item), 2)))
    new += ':00'
    val4.append(new)

index = val4.index('24:00:00')
val4[index] = '00:00:00'

dic4 = dict(zip(char4,val4))

val3 = []
for item in char3:
    filled = item.zfill(4)
    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
    new += ':00'
    val3.append(new)
    
dic3 = dict(zip(char3,val3))

val2 = []
for item in char2:
    filled = item.zfill(4)
    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
    new += ':00'
    val2.append(new)
    
dic2 = dict(zip(char2,val2))

val1 = []
for item in char1:
    filled = item.zfill(4)
    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
    new += ':00'
    val1.append(new)
    
dic1 = dict(zip(char1,val1))

In [62]:
# Combine dictionaries to create one central dict

rep_dict = {}
for dic in [dic1, dic2, dic3, dic4, dic5]:
    rep_dict.update(dic)


In [63]:
rep_dict

{'2': '00:02:00',
 '5': '00:05:00',
 '7': '00:07:00',
 '10': '00:10:00',
 '12': '00:12:00',
 '22': '00:22:00',
 '23': '00:23:00',
 '24': '00:24:00',
 '000': '00:00:00',
 '030': '00:30:00',
 '100': '01:00:00',
 '200': '02:00:00',
 '230': '02:30:00',
 '300': '03:00:00',
 '400': '04:00:00',
 '515': '05:15:00',
 '530': '05:30:00',
 '545': '05:45:00',
 '615': '06:15:00',
 '620': '06:20:00',
 '630': '06:30:00',
 '635': '06:35:00',
 '640': '06:40:00',
 '645': '06:45:00',
 '650': '06:50:00',
 '700': '07:00:00',
 '710': '07:10:00',
 '715': '07:15:00',
 '725': '07:25:00',
 '730': '07:30:00',
 '732': '07:32:00',
 '735': '07:35:00',
 '740': '07:40:00',
 '745': '07:45:00',
 '750': '07:50:00',
 '800': '08:00:00',
 '805': '08:05:00',
 '810': '08:10:00',
 '815': '08:15:00',
 '830': '08:30:00',
 '845': '08:45:00',
 '850': '08:50:00',
 '900': '09:00:00',
 '915': '09:15:00',
 '930': '09:30:00',
 '945': '09:45:00',
 '0000': '00:00:00',
 '0010': '00:10:00',
 '0015': '00:15:00',
 '0026': '00:26:00',
 '0030'

In [64]:
# Function taken from stackoverflow: https://stackoverflow.com/questions/6116978/how-to-replace-multiple-substrings-of-a-string
# Note: this function takes quite a while to run
# Biggest issue to deal with here is multiple replacements on one string - which this function seems to solve

import re

def multiple_replace(string):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)], string)

df['value_time_event'] = df.apply(lambda x: multiple_replace(x['value']) if x['value'] is not np.nan else np.nan, axis=1)

In [65]:
# Where 'value_time_event' is null - fill in with 00:00:00
df["value_time_event"].fillna(value = "00:00:00", inplace = True) 

In [66]:
# Check format of numbers in value column based on 'risetime' and 'bedtime' events

bool_series = df['event2'] == 'risetime'
df[bool_series]['value_time_event'].unique()

# From the output - we can see that 'risetime' events work

array(['09:00:00', '08:30:00', '12:00:00', '10:15:00', '11:15:00',
       '11:00:00', '08:50:00', '00:00:00', '10:00:00', '08:00:00',
       '06:30:00', '07:45:00', '00:10:00', '09:30:00', '06:45:00',
       '08:15:00', '07:30:00', '07:00:00', '07:40:00', '07:35:00',
       '08:45:00', '06:00:00', '08:25:00', '07:15:00', '06:35:00',
       '06:15:00', '09:15:00', '05:15:00', '08:35:00', '11:20:00',
       '07:50:00', '06:40:00', '08:40:00', '05:45:00', '08:05:00',
       '05:30:00', '13:00:00', '10:30:00', '11:35:00', '08:20:00',
       '06:50:00', '09:45:00', '07:10:00', '06:20:00', '15:52:00',
       '00:05:00', '15:56:00', '18:25:00', '16:52:00', '09:50:00',
       '07:52:00', '07:27:00', '07:38:00', '00:07:00', '08:10:00',
       '11:30:00', '04:00:00', '12:34:00', '12:35:00', '11:01:00',
       '07:22:00', '08:04:00', '07:25:00', '08:31:00', '10:06:00',
       '08:34:00', '08:46:00', '07:53:00', '07:57:00', '08:19:00',
       '07:34:00', '05:52:00', '06:38:00', '05:53:00', '09:20:

In [67]:
# Check format of numbers in value column based on 'risetime' and 'bedtime' events

bool_series = df['event2'] == 'bedtimetonight'
df[bool_series]['value_time_event'].unique()

array(['23:00:00', '12:00:00', '23:30:00', '22:00:00', '00:00:00',
       '01:00:00', '00:23:00', '00:30:00', '22:30:00', '11:30:00',
       '23:45:00', '02:30:00', '01:30:00', '22:45:00', '11:45:00',
       '02:00:00', '23:15:00', '03:00:00', '00:02:00', '02:24:54',
       '03:30:00', '00:22:00', '23:40:00', '00:19:00', '11:20:00',
       '15:57:00', '00:05:00', '15:53:00', '14:23:00', '16:53:00',
       '00:10:00', '23:59:00', '23:50:00', '12:36:00', '12:34:00',
       '12:37:00', '00:15:00', '04:00:00', '00:24:00', '22:15:00',
       '00:40:00', '00:12:00', '22:50:00', '00:36:00', '22:40:00',
       '00:26:00', '23:05:00'], dtype=object)

In [68]:
# Replace 'time_event_id' with 'value_time_event' value where event2 == 'bedtimetonight' or event2 == 'risetime'

df['time_event_id'] = np.where(df['event2'] == 'bedtimetonight', df['value_time_event'], df['time_event_id'])
df['time_event_id'] = np.where(df['event2'] == 'risetime', df['value_time_event'], df['time_event_id'])

In [69]:
# Combining date and time into one column

df['date_time'] = (df['date_event_id'] +" "+ df['time_event_id']).astype(str)
df.replace('nan', np.nan, regex=True, inplace = True)
df.dtypes
df

,row_id,user_id,event_id,value,date_event_id,time_event_id,event_id_clean,event1,event2,value_time_event,date_time
0,1,10,lamp_change_29_mei_2015_19_08_33_984,OFF,29 mei 2015,19:08:33,"lamp change , mei",lamp change,mei,OFF,29 mei 2015 19:08:33
1,2,10,0010_31_mei_2015_bedtime_tonight,2300,31 mei 2015,23:00:00,"mei , bedtime tonight",mei,bedtimetonight,23:00:00,31 mei 2015 23:00:00
2,3,10,0010_31_mei_2015_fitness,52,31 mei 2015,00:00:00,"mei , fitness",mei,fitness,00:05:0000:02:00,31 mei 2015 00:00:00
3,4,10,morning_backup_minute,0,NaN,00:00:00,"morning backup , minute",morning backup,minute,0,NaN
4,5,10,lamp_change_29_mei_2015_19_08_33_942,OFF,29 mei 2015,19:08:33,"lamp change , mei",lamp change,mei,OFF,29 mei 2015 19:08:33
...,...,...,...,...,...,...,...,...,...,...,...
1134006,1134007,63,error_event_15_september_2015_20_33_31_287,1_lamps_found,15 september 2015,20:33:31,"error event , september",error event,september,1_lamps_found,15 september 2015 20:33:31
1134007,1134008,63,error_event_16_september_2015_21_15_35_568,0_lamps_found,16 september 2015,21:15:35,"error event , september",error event,september,0_lamps_found,16 september 2015 21:15:35
1134008,1134009,63,start_experiment,2015-09-06T00:00:00.000+02:00,NaN,00:00:00,start experiment,start experiment,NaN,00:02:000100:05:00-09-06T00:00:00.00:00:00+000...,NaN
1134009,1134010,63,lamp_change_09_september_2015_23_39_52_045,OFF,09 september 2015,23:39:52,"lamp change , september",lamp change,september,OFF,09 september 2015 23:39:52


In [71]:
# Creating the date column using datetime.datetime as instructed in the Hints

def date_convert(date_to_convert):
    return datetime.datetime.strptime(date_to_convert, '%d %B %Y %H:%M:%S')

df['date'] = df.apply(lambda x: date_convert(x['date_time']) if x['date_time'] is not np.nan else np.nan, axis=1)

In [77]:
# Fitler only informative events

key_words = ['nudge_time', 'lamp_change', 'bedtime_tonight', 'risetime', 'rise_reason', 
                'adherence importance', 'fitness']

key_regex = '.*(' + '|'.join(key_words) + ')'
new_df = df.loc[df.event_id.str.match(key_regex)]

In [78]:
# check if there are any null values in the filtered df

bool_series = pd.isnull(new_df['date'])
new_df[bool_series]

,row_id,user_id,event_id,value,date_event_id,time_event_id,event_id_clean,event1,event2,value_time_event,date_time,date


In [ ]:
# set index to date, user_id
df.set_index(['date', 'user_id'], inplace=True)

In [ ]:
# check the index
print(df.index)


In [ ]:
df.head(50)

In [ ]:
# check the bedtime column
display(df[['bedtime']])


In [ ]:
# check the intended_bedtime column
display(df[['intended_bedtime']])


In [ ]:
# check the rise_time
display(df[['rise_time']])


In [ ]:
# check the rise_reason, fitness, adherence_importance column
display(df[['rise_reason', 'fitness', 'adherence_importance']])


In [ ]:
# check the in_experimental_group column
display(df[['in_experimental_group']])


## Exercise 2 (10 + 20 points)
The second part of this assignment is to store the contents of the DataFrame into MongoDB, and to write a function that retrieves data from MongoDB and outputs it in a user-friendly format.

<ol>
<li>
The data should be stored in the collection "sleepdata" in the database "BigData". Make sure to use
the same column names as specified for the DataFrame, and to define the correct primary key. See
the document with the "Tips" for some comments about the primary key. Add the extra columns "date",
"user", "sleep duration" to facilitate sorting the data if necessary. Here, "sleep duration" is the difference between the risetime and the bedtime.
<br><br></li>
<li>The following is an example of how the output must be presented.
    
| date | user | bedtime | intended | risetime | reason | fitness | adh | in_exp | sleep_duration |
-------|------|---------|----------|----------|--------|---------|-----|--------|----------------|    
| 11-06-2015 | 2  | 00:51:28 | 22:30:00 | 07:00:00 | ja  | -    | 47.0 | no  | 22351 |
| 11-06-2015 | 20 | 00:28:10 | 23:00:00 | 07:10:00 | nee | 55.0 | 88.0 | yes | 33510 |
| 11-06-2015 | 34 | 19:54:10 | -        | -        | -   | -    | -    | yes | -     |

Here sleep duration is in number of seconds. Note that, in order to determine the sleep duration of day X, it
is necessary to know the risetime of day X, but the bedtime of day X - 1.
<br><br></li>
</ol>

In [ ]:
def to_mongodb(df):
# YOUR CODE HERE

    # connect to MongoDB database
    client = pymongo.MongoClient("localhost", 27017)
    db = client.BigData
    sleepdata = db.sleepdata
    
    sleepdata.delete_many({})
    
# YOUR CODE ENDS HERE

In [ ]:
to_mongodb(df)


In [ ]:
def read_mongodb(filter,sort):
# YOUR CODE HERE

    # connect to MongoDB database
    connection = pymongo.MongoClient("localhost", 27017)
    db = connection.BigData
    sleepdata = db.sleepdata

# YOUR CODE ENDS HERE

In [ ]:
query = read_mongodb({'sleep_duration': {'$gt': 40000}}, '_id')
print(query)
